# Torchtext 라이브러리 텍스트 분류
- 1단계. 데이터 전처리 : 숫자형식으로 변환하는 것까지
- 2단계. 모델 구현
- [1-1] 데이터 준비 => 내장 데이터셋 활용 => 내장 데이터셋 활용

In [1]:
import torch
from torchtext.datasets import AG_NEWS

# DATA pipe 타입 > iterator 타입 형변환
train_iter = iter(AG_NEWS(split='train'))

In [2]:
# 데이터 확인 => (label, text), label 1~4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

# 데이터 처리 파이프라인 준비
- 어휘집 vocab, 단어 벡터word vector, 토크나이저 tokenizer
- 가공되지 않은 문자열에 대한 데이터 처리 빌딩 블록
- 일반적인 NLP 데이터 처리
    - 첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성
        => 토큰 목록 또는 반복자 받는 내장 팩토리 함수(factory function): build_vocab_from_iterator
    - 사용자는 어휘집에 추가할 특수 기호(special symbol) 전달 가능

In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# 토커나이저 생성
tokenizer = get_tokenizer("basic_english")

# 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split='train')

In [5]:
# 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트가 나옴 -> 텍스트만 토큰화하면 됨 
        yield tokenizer(text)

In [7]:
# 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

# unk 인덱스 0으로 설정
vocab.set_default_index(vocab["<unk>"])

In [8]:
vocab(["<unk>", "here", "is", "an", "example"])

[0, 475, 21, 30, 5297]

In [9]:
# 텍스트 -> 정수 인코딩
text_pipeline = lambda x: vocab(tokenizer(x)) # 다 쪼갠게 들어감

# 레이블 -> 정수 인코딩
label_pipeline = lambda x: int(x) -1 # 레이블=1~4를 0~3으로 만드는 것

# 3. 데이터 배치와 반복자 생성
- torch.utils.data.DataLoader : getitem(), len() 구현한 맵 형태(map-style)
- collate_fn(): DataLoader로부터 생성된 샘플 배치 함수
    - 입력 : DataLoader에 배치 크가가 있는 배치 데이터

In [10]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 배치 크기 만큼 데이터셋 반환 함수
def collate_batch(batch):
    # 배치크기 만큼의 라벨, 텍스트, 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0] # offset은 글자마다 길이가 다르니까 해당 글자의 길이에 대한 정보를 주는 것
    
    # 1개씩 뉴스기사, 라벨 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))
        
        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        
        # 텍스트 offset 즉, 텍스트 크기/길이 저장 
        offsets.append(processed_text.size(0))
    
    # 텐서화
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [11]:
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, 
                        batch_size=8,
                        shuffle=False, 
                        collate_fn=collate_batch)

In [13]:
# 분류 클래스 수와 단어 사전 개수
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f"num_class : {num_class}\nvocab_size : {vocab_size}")

C:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


num_class : 4
vocab_size : 95811


In [16]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets) 
    break

tensor([2, 2, 2, 2, 2, 2, 2, 2]) tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27,    14,    27,   